In [1]:
import os
from pathlib import Path
import shutil
from urllib.parse import parse_qs

import ipywidgets as widgets
from IPython.display import display, clear_output

import utils

In [2]:
### Settings

CLASSES = ['Healthy', 'OC Degen. Variant 2', 'OC Degen. Variant 1'] 
input_stages = {'0': 'training', '1': 'validation_nodiag', '2': 'validation_noxai'} # used for obscuring the stage names
seeds = [23, 45, 6] # randomly selected values to seed file/example ordering. one per each stage

# parse notebook url to get prolific id if provided
query_string = os.environ.get('QUERY_STRING', '')
parameters = parse_qs(query_string)
url_pid = parameters.get('pid', [''])[0]
stage = parameters.get('stage', '0')[0]
seed  = seeds[int(stage)]

# get stage info
if stage not in input_stages.keys():
		raise ValueError(f'Invalid stage number: {stage}')
stage = input_stages[stage]

# setup parameters for interface
include_diagnoses = stage != 'validation_nodiag'
include_truth = stage == 'training'
n_samples = 15 if stage =='training' else 39
# id_len = 5  # TODO: change this when ready to publish
id_len = 24 # prolific ids are 24 alphanumeric chars

In [3]:
# setup folder paths
oc_basedir = Path(f'1. Research/1. HCXAI/1. Projects/evalxai_studies/example_validation_study/{stage}')
tmp_folder = utils.create_tmp_folder(stage)

In [4]:
# Text for the various HTML Elements
cdown_text = '<strong style="font-size: 1.25em">Diagnosis - {count:02} / {n_samples} </strong>'

diagnosis_text = "<h3> Patient ID: <i>{disp_id}</i> </h3>"
if include_diagnoses:
	diagnosis_text += "<h3>AI Diagnosis Recommendation:</br><i style='color: rgb(25, 118, 210);'>{diagnosis}</i> </h3>"
	
if include_truth:
  diagnosis_text += "<h3>True Diagnosis:</br><i style='color: rgb(44, 160, 44);'>{actual}{attrs}</i> </h3>"

disclaimer_text = f"""<h3 style="margin-bottom: 0.05em;">Warning:</h3><p style='font-size=8px; line-height: 1.1em; margin-top: 0.05em'><i>Treating OCDegen is costly and highly invasive. 
                     Please verify all diagnoses before making your final decision. Furthermore, incorrect reponses negatively affect your performance bonus.</i></p>"""

success_text = """<div style='text-align: center;'><h3 style='margin-bottom: 0px;'>Thank you for your diagnoses.</h3>
                  <p style='margin-top: 0px;'>Please enter the following code into LimeSurvey:</p>
                  <h3 style='color: green;'><i>{hash}</i></h3></div>"""

selector_text = '<b>Your Diagnosis:</b>'

description_text = f"""
<div style="display: flex; font-family: Arial, sans-serif; height: 100%; width: 100%;">
    <div style="flex: 0.85; background-color: #f9f9f9; padding: 1em; border-right: 3px solid #ccc; box-sizing: border-box; height: 100%;">
        <h2 style="text-align: center; font-size: 1.4em; margin-bottom: 1em; margin-top: 0em; line-height: 1.3em;">OC Degeneration Biomarkers</h2>
        
        <div style="margin-bottom: 1em;">
            <h3 style="margin-bottom: 0.3em; font-size: 1.2em;"> Variant 1</h3>
            <ol style="padding-left: 1.5em; margin: 0; line-height: 1.2; font-size: 1em; color: #555;">
                <li style="margin-bottom: 0;">Medium Spine Bend</li>
                <li style="margin-bottom: 0;">Strong Bone Variation</li>
                <li style="margin-bottom: 0;">Tucked Head</li>
                <li style="margin-bottom: 0;">Color Mutation</li>
            </ol>
        </div>
        
        <div>
            <h3 style="margin-bottom: 0.3em; font-size: 1.2em;"> Variant 2</h3>
            <ol style="padding-left: 1.5em; margin: 0; line-height: 1.2; font-size: 1em; color: #555;">
                <li style="margin-bottom: 0;">Strong Spine Bend</li>
                <li style="margin-bottom: 0;">Medium Bone Variation</li>
                <li style="margin-bottom: 0;">Mutation of Main Bones</li>
                <li style="margin-bottom: 0;">Color Mutation</li>
            </ol>
        </div>
        <h3 style="font-size: 1.2em; margin-bottom: 0.05em; margin-top: 2em;">Warning:</h3><p style='font-size=1em; line-height: 1.1em; margin-top: 0.0em'><i>Treating OCDegen is <strong>costly and highly invasive</strong>. 
                     Please verify all diagnoses before making your final decision. Furthermore, <strong>incorrect reponses negatively affect your performance bonus.</strong></i></p>
    </div>
</div>
"""

In [5]:
results = {}

def display_submit_button(pro_id, app, back_button, next_button, selector):
  # Create a submit button
  submit_button = widgets.Button(description="Submit Diagnoses",
                                  icon='check',
                                  disabled=True,
                                  layout = widgets.Layout(height='45px', width='auto', 
                                                          align_content='center', justify_content='center'))
  submit_button.style.button_color = 'lightblue'

  # register new change for the submit button
  def on_selector_change2(change):
    global e_idx
    global examples 

    # check if the last example is none to enable submit button
    if change['type'] == 'change' and change['name'] == 'value':
      last_selection = results[examples[-1].id].select
      if last_selection is None:
        submit_button.disabled = True
      else:
        submit_button.disabled = False

      # now if last example we must activate submit button since there is no next button
      if e_idx == len(examples) - 1 and selector.index is not None:
        submit_button.disabled = False
        

  selector.observe(on_selector_change2, names='value')

  def on_button_clicked(b):
    global e_idx
    global examples 

    # update the last example if it has changed
    if selector.index != results[examples[e_idx].id].select:
      results[examples[e_idx].id].set_select(selector.index)

    try:
      # disable buttons while saving
      submit_button.disabled = True
      submit_button.button_style = 'success'

      back_button.disabled = True
      back_button.button_style = 'success'
      next_button.disabled = True
      next_button.button_style = 'success'
      selector.disabled = True

      df = utils.save_results(results, pro_id, oc_basedir, tmp_folder, stage)

      # remove folder after files transfered and update button with success
      shutil.rmtree(tmp_folder)
      submit_button.description = 'Submission Successful'

      # Replace all buttons with success message and hash code
      hash = utils.hash_prof_id(pro_id, stage=stage)
      success_html = widgets.HTML(
        value = success_text.format(hash=hash),
        layout = widgets.Layout(align_content='center', justify_content='center')
      )
      app[5:7, 1:3] = success_html

    except Exception as e:
      # if submission doesn't work enable buttons
      submit_button.disabled = False
      submit_button.button_style = 'success'

      back_button.disabled = False
      back_button.button_style = 'success'
      next_button.disabled = False
      next_button.button_style = 'success'
      selector.disabled = False

      # need to fix this...
      success_html.value = f'Submission not successfully. Try again.<br/> {e}'
      app[6:7, 1:3] = success_html

  submit_button.on_click(on_button_clicked)

  app[7:8, 1:3] = submit_button

def start_eval(image_paths, df_samples, pro_id, out):

  global e_idx
  global pbar
  global examples
  global results
  global submit_showing

  e_idx = 0
  examples = utils.setup_examples(image_paths, df_samples, seed=seed, start_size=8)
  e = examples[e_idx]

  submit_showing = False

  # setup image
  file = open(e.image_path, "rb")
  image = file.read()
  image_display = widgets.Image(
    value=image,
    width=256,
    height=256,
  )
  image_box = widgets.HBox([image_display], layout=widgets.Layout(justify_content='center', align_content='center'))

  # HTML
  example_text = diagnosis_text
  
  attrs = f'<br />Based on biomarkers {e.ill_chars}' if e.true !=0 else ''
  example_html = widgets.HTML(
    value=example_text.format(disp_id=e.disp_id, diagnosis=CLASSES[e.pred], actual=CLASSES[e.true], attrs=attrs)
  )
  example_html.layout = widgets.Layout(align_self='flex-start')
  
  disclaimer_html = widgets.HTML(
    value=disclaimer_text,
    layout=widgets.Layout(
      height='100%',  # Adjust as needed
      width='100%',    # Adjust as needed
    )
  )
  description_html = widgets.HTML(
    value=description_text,
    layout=widgets.Layout(
      height='500px',  # Adjust as needed
      width='100%',    # Adjust as needed
    )
  )

  cdown_html = widgets.HTML(
    value= cdown_text.format(count=e_idx+1, n_samples=n_samples)
  )
  pbar = widgets.IntProgress(min=0, max=n_samples-1,
                             layout=widgets.Layout(justify_content='center', align_content='center'))
  pbar_box = widgets.HBox([cdown_html, pbar], layout=widgets.Layout(justify_content='center', align_content='center'))
  # pbar_box = widgets.HBox([widgets.Label(value="The $m$ in $E=mc^2$:"), pbar])
  

  # setup selector
  opts_map = {CLASSES[0]: 0, CLASSES[2]: 2, CLASSES[1]: 1}
  opts = list(opts_map.keys())
  # random.shuffle(opts)

  # hack to update Radio Buttons label to bold
  display(widgets.HTML("""
  <style>
  .radio-style .widget-label {
      font-size: 15px !important;
      font-weight: bold !important;
  }
  </style>
  """))
  selector = widgets.RadioButtons(
      options=opts,
      value=None,
      description='Select Your Diagnosis:',
      disabled=False,
      layout = widgets.Layout(width='auto'),
      _dom_classes=["radio-style"]
  )
  selector_box = widgets.HBox([selector], layout=widgets.Layout(justify_content='center', align_content='center'))

  # selector.style.button_color = 'lightblue'

  # _style = widgets.HTML(
  # 	"<style>.widget-radio-box {flex-direction: row !important;}.widget-radio-box"
  # 	" label{margin:5px !important;width: 120px !important;} .widget-label{text-align: center !important;}</style>",
  # 	layout=widgets.Layout(display="none"),
  # )
  # selector_horizontal = widgets.HBox([selector, _style])

  results = utils.register_example(results, e, opts_map[selector.value] if selector.value is not None else None)

  # setup Back Button
  back_button = widgets.Button(description="Previous",
                               disabled=True,
                               icon='step-backward',
                               layout=widgets.Layout(height='auto', width='auto'))
  back_button.style.button_color = 'lightblue'

  # setup Next Button
  next_button = widgets.Button(description="Next",
                               icon='step-forward',
                               layout=widgets.Layout(height='auto', width='auto'),
                               disabled=True)
  next_button.style.button_color = 'lightblue'

  # define callbacks
  def on_back_clicked(e):
    global e_idx
    global pbar
    global examples
    global results

    # update the last example if it has changed
    if selector.index != results[examples[e_idx].id].select:
      results[examples[e_idx].id].set_select(selector.index)

    # move to the previous example
    e_idx -= 1
    pbar.value -= 1
    cdown_html.value = cdown_text.format(count=e_idx+1, n_samples=n_samples)

    back_button.disabled = False if e_idx > 0 else True
    next_button.disabled = False if e_idx < len(examples) else True

    if e_idx >= 0:
      e = examples[e_idx]
      if e.id in results:
        selector.index = results[e.id].load()
      else:
        raise LookupError(f'Result should exist for id {e.id}')
      
      attrs = f'<br />Based on biomarkers {e.ill_chars}' if e.true !=0 else ''
      example_html.value = example_text.format(disp_id=e.disp_id, diagnosis=CLASSES[e.pred], actual=CLASSES[e.true], attrs=attrs)

      file = open(e.image_path, "rb")
      image = file.read()
      image_display.value = image 

  back_button.on_click(on_back_clicked)

  def on_selector_change(change):
    global e_idx
    global examples 

    # id = examples[e_idx].id

    # if change['type'] == 'change' and change['name'] == 'value':
    #   results[id]['select'] = opts_map[change['new']] if change['new'] is not None else None

    if e_idx == len(examples) - 1:
      next_button.disabled = True
    else:
      next_button.disabled = False

  
  selector.observe(on_selector_change, names='value')

  def on_next_clicked(b):
    global e_idx
    global pbar
    global examples
    global results 
    global submit_showing

    # update the last example if it has changed
    if selector.index != results[examples[e_idx].id].select:
      results[examples[e_idx].id].set_select(selector.index)

    # move to the next example
    e_idx += 1
    pbar.value += 1
    cdown_html.value = cdown_text.format(count=e_idx+1, n_samples=n_samples)

    back_button.disabled = False if e_idx > 0 else True

    if e_idx < len(examples):
      e = examples[e_idx]
      if e.id not in results:
        # new sample that has not yet been diagnosed
        results = utils.register_example(results, e, None)
        selector.value = None
        next_button.disabled = True
      else:
        # existing sample that has been diagnosed
        selector.index = results[e.id].load()

        # if exists it could be none (if back button was used)
        if results[e.id].select is None:
          next_button.disabled = True
        else:
          next_button.disabled = False

      attrs = f'<br />Based on biomarkers {e.ill_chars}' if e.true !=0 else ''
      example_html.value = example_text.format(disp_id=e.disp_id, diagnosis=CLASSES[e.pred], actual=CLASSES[e.true], attrs=attrs)
      file = open(e.image_path, "rb")
      image = file.read()
      image_display.value = image
    
    if e_idx == len(examples) - 1:
      next_button.disabled = True

      if not submit_showing:
        with out:
          # clear_output(wait=True)
          display_submit_button(pro_id, app, back_button, next_button, selector)
          submit_showing = True

  next_button.on_click(on_next_clicked)

  # setup the App
  app = widgets.GridspecLayout(8, 4,
                               height='600px',
                               width='1200px',
                               align_items='center', 
                               justify_items='center')

  # app[0, 1:2] = cdown_html
  app[0, 1:3] = pbar_box
  app[:7, 0:1] = description_html
  app[1:4, 1:2] = image_box
  app[1:3, 2:] = example_html
  # app[4:6, 1:2] = disclaimer_html
  app[4:6, 1:3] = selector_box
  app[6, 1] = back_button
  app[6, 2] = next_button
  
  display(app)


def start(image_paths, df_samples):

  global results
  results = {}
    
  html = widgets.HTML(
    value=f"<div style='padding-top: 10px;'></div><h1>Enter Prolific ID.</h1>",
  )
  display(html)

  error_html = widgets.HTML(
    value=''
  )
  
  id_field = widgets.Text(
    description='Prolific ID',
    value=url_pid,
    placeholder='Enter your prolific survey ID...',
    disable=False
  )

  next_button = widgets.Button(description="Start",
                               icon='play')
  next_button.style.button_color = 'lightblue'

  out = widgets.Output()

  def on_text_change(change):
    error_html.value = ''

  def on_button_clicked(b):
    if utils.results_exist(id_field.value, oc_basedir, tmp_folder, stage):
      error_html.value = "<div style='color: red;'><i>Prolific ID already exists</i>.  Please try reentering your ID from Prolific.</div>"
    elif not (utils.id_valid(id_field.value, id_len=id_len)):
      error_html.value = "<div style='color: red;'><i>Not a valid Prolific ID</i>. Please try reentering your ID from Prolific.</div>"
    else:
      next_button.button_style = 'success'
      next_button.disabled = True
      id_field.disabled = True

      id_field.close()
      html.close()
      b.close()
    
      with out:
        clear_output(wait=True)
        start_eval(image_paths, df_samples, id_field.value, out)

  id_field.observe(on_text_change, names='value')
  display(id_field)

  next_button.on_click(on_button_clicked)
  display(next_button)
  display(error_html)
  display(out)

In [6]:
start(*utils.init(oc_basedir, tmp_folder, num_samples=n_samples))

HTML(value="<div style='padding-top: 10px;'></div><h1>Enter Prolific ID.</h1>")

Text(value='', description='Prolific ID', placeholder='Enter your prolific survey ID...')

Button(description='Start', icon='play', style=ButtonStyle(button_color='lightblue'))

HTML(value='')

Output()

KeyError: 'attrs'

In [41]:
# next steps
# 5. Attention check example
# - Fix submit error interface...
# - Load testing

# done
# Progress bar
# basic app layout
# Validate input id and fix check for existing results
# implement hashing for success code to enter to lime survey
# implements URL parsing to enable one app for multiple XAI studies
# 8. Use URL parsing to set params for including diagnoses or not? but should be obsure?
#    (only if i can get prof id from lime survey)
# 7. Fix temp folder creation when created at same second - added MS this should work hopefully
# 6. Testing - verify results match ids - validated ids, ground truth and predictions via Google Colab